In [ ]:
from sklearn.datasets import fetch_20newsgroups         # Dataset che contiene testo di articoli di giornale appartenenti a 20 categorie differenti
import nltk                                             # NLP toolkit
import re                                               # Libreria per operazioni con le espressioni regolari
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import itertools
import random

nltk.download('punkt')                                  # Con questo comando si scarica il tokenizzatore 'Punkt'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Scarichiamo il nostro dataset

In [ ]:
train_news_texts, category = fetch_20newsgroups(subset="train", categories=["sci.space"], return_X_y=True, remove=['headers', 'footers', 'quotes'])

## Definiamo la funzione creata in precedenza per processare il dataset e creare gli N-Grammi

In [ ]:
def preprocessing(text, n):
    """
    Funzione che preprocessa il testo per creare una frase tokenizzata.

    Args:
        text: stringa contenente il testo da preprocessare e tokenizzare

    Returns:
        testo preprocessato e tokenizzato
    """
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9.?! ]+', '', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text_tokenized = nltk.word_tokenize(text)
    text_tokenized = ['<s>'] * (n - 1) + text_tokenized + ['</s>']
    return text_tokenized


In [ ]:
def sentence_to_ngram(tokenized_sentence, n=3):
    """
    Funzione che restituisce tutti i n-grammi contenuti all'interno della frase tokenizzata.

    Args:
        tokenized_sentence: lista di parole/tokens che compongono la frase
        n: n-grammi da considerare

    Returns:
        lista di tutti i n-grammi presenti all'interno della frase tokenizzata
    """

    ngram_list = []
    for i in range(len(tokenized_sentence) - n + 1):
        # the sliding window starts at position i and contains 3 words
        ngram = tokenized_sentence[i : i + n]
        ngram_list.append(ngram)
    return ngram_list



# Prendiamo due frasi come esempio

In [ ]:
n = 3
train_news_texts_sample = [nltk.sent_tokenize(train_news_texts[0].strip())[0], nltk.sent_tokenize(train_news_texts[0].strip())[1],  nltk.sent_tokenize(train_news_texts[0].strip())[0]]
for text in train_news_texts_sample:
  print(text)
  print("-"*80)


Any lunar satellite needs fuel to do regular orbit corrections, and when
its fuel runs out it will crash within months.
--------------------------------------------------------------------------------
The orbits of the Apollo
motherships changed noticeably during lunar missions lasting only a few
days.
--------------------------------------------------------------------------------
Any lunar satellite needs fuel to do regular orbit corrections, and when
its fuel runs out it will crash within months.
--------------------------------------------------------------------------------


# Creiamo il corpus e definiamo il vocabolario con le 10 parole più comuni


In [ ]:
train_news_texts_sample_processed = [preprocessing(text, n) for text in train_news_texts_sample]
corpus = [x for y in train_news_texts_sample_processed for x in y]
corpus

['<s>',
 '<s>',
 'any',
 'lunar',
 'satellite',
 'needs',
 'fuel',
 'to',
 'do',
 'regular',
 'orbit',
 'corrections',
 'and',
 'when',
 'its',
 'fuel',
 'runs',
 'out',
 'it',
 'will',
 'crash',
 'within',
 'months',
 '.',
 '</s>',
 '<s>',
 '<s>',
 'the',
 'orbits',
 'of',
 'the',
 'apollo',
 'motherships',
 'changed',
 'noticeably',
 'during',
 'lunar',
 'missions',
 'lasting',
 'only',
 'a',
 'few',
 'days',
 '.',
 '</s>',
 '<s>',
 '<s>',
 'any',
 'lunar',
 'satellite',
 'needs',
 'fuel',
 'to',
 'do',
 'regular',
 'orbit',
 'corrections',
 'and',
 'when',
 'its',
 'fuel',
 'runs',
 'out',
 'it',
 'will',
 'crash',
 'within',
 'months',
 '.',
 '</s>']

In [ ]:
word_count = Counter(corpus)
sorted(word_count.items(), key=lambda pair: pair[1], reverse=True)

[('<s>', 6),
 ('fuel', 4),
 ('lunar', 3),
 ('.', 3),
 ('</s>', 3),
 ('any', 2),
 ('satellite', 2),
 ('needs', 2),
 ('to', 2),
 ('do', 2),
 ('regular', 2),
 ('orbit', 2),
 ('corrections', 2),
 ('and', 2),
 ('when', 2),
 ('its', 2),
 ('runs', 2),
 ('out', 2),
 ('it', 2),
 ('will', 2),
 ('crash', 2),
 ('within', 2),
 ('months', 2),
 ('the', 2),
 ('orbits', 1),
 ('of', 1),
 ('apollo', 1),
 ('motherships', 1),
 ('changed', 1),
 ('noticeably', 1),
 ('during', 1),
 ('missions', 1),
 ('lasting', 1),
 ('only', 1),
 ('a', 1),
 ('few', 1),
 ('days', 1)]

In [ ]:
M = 10

vocabulary = Counter(word_count).most_common(M)

# Teniamo nel nostro vocabolario solamente le parole, senza il conteggio di quante volte compaiono
vocabulary = [word for word, frequency in vocabulary]
# Siccome abbiamo delle parole non presenti nel nostro vocabolario, aggiungiamoci anche il token UNK (e se non presenti già, i token <s> e </s>)
vocabulary.append('UNK')


In [ ]:
vocabulary

['<s>',
 'fuel',
 'lunar',
 '.',
 '</s>',
 'any',
 'satellite',
 'needs',
 'to',
 'do',
 'UNK']

In [ ]:
sentence = ['any','lunar','satellite','needs','fuel','to','do','regular','orbit','corrections']
output_sentence = []
print(f"input sentence: {sentence}")

for w in sentence:
    # test if word w is in vocabulary
    if w in vocabulary:
        output_sentence.append(w)
    else:
        output_sentence.append('UNK')

print(f"output sentence: {output_sentence}")


input sentence: ['any', 'lunar', 'satellite', 'needs', 'fuel', 'to', 'do', 'regular', 'orbit', 'corrections']
output sentence: ['any', 'lunar', 'satellite', 'needs', 'fuel', 'to', 'do', 'UNK', 'UNK', 'UNK']


# Definiamo le tecniche per modellare anche la presenza di token non contenuti nel nostro vocabolario

## Smoothing

In [ ]:
def add_k_smooting_probability(k, vocabulary_size, n_gram_count, n_gram_prefix_count):
    numerator = n_gram_count + k
    denominator = n_gram_prefix_count + k * vocabulary_size
    return numerator / denominator

trigram_probabilities = {('any', 'lunar', 'satellite') : 3}
bigram_probabilities = {( 'any', 'lunar') : 10}
vocabulary_size = 10
k = 3

probability_known_trigram = add_k_smooting_probability(k, vocabulary_size, trigram_probabilities[('any', 'lunar', 'satellite')],
                           bigram_probabilities[( 'any', 'lunar')])

probability_unknown_trigram = add_k_smooting_probability(k, vocabulary_size, 0, 0)

print(f"probability_known_trigram: {probability_known_trigram}")
print(f"probability_unknown_trigram: {probability_unknown_trigram}")


probability_known_trigram: 0.15
probability_unknown_trigram: 0.1


## Back-off

In [ ]:
# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('any', 'lunar', 'satellite'): 0}
bigram_probabilities = {( 'lunar', 'satellite'): 0.3}
unigram_probabilities = {'satellite': 0.4}

# this is the input trigram we need to estimate
trigram = ('large', 'mars', 'satellite')

# find the last bigram and unigram of the input
bigram = trigram[1: 3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# 0.4 is used as an example, experimentally found for web-scale corpuses when using the "stupid" back-off
lambda_factor = 0.4
probability_hat_trigram = 0

# search for first non-zero probability starting with trigram
# to generalize this for any order of n-gram hierarchy,
# you could loop through the probability dictionaries instead of if/else cascade
if trigram not in trigram_probabilities or trigram_probabilities[trigram] == 0:
    print(f"probability for trigram {trigram} not found")

    if bigram not in bigram_probabilities or bigram_probabilities[bigram] == 0:
        print(f"probability for bigram {bigram} not found")

        if unigram in unigram_probabilities:
            print(f"probability for unigram {unigram} found\n")
            probability_hat_trigram = lambda_factor * lambda_factor * unigram_probabilities[unigram]
        else:
            probability_hat_trigram = 0
    else:
        probability_hat_trigram = lambda_factor * bigram_probabilities[bigram]
else:
    probability_hat_trigram = trigram_probabilities[trigram]

print(f"probability for trigram {trigram} estimated as {probability_hat_trigram}")


besides the trigram ('large', 'mars', 'satellite') we also use bigram ('mars', 'satellite') and unigram (satellite)

probability for trigram ('large', 'mars', 'satellite') not found
probability for bigram ('mars', 'satellite') not found
probability for unigram satellite found

probability for trigram ('large', 'mars', 'satellite') estimated as 0.06400000000000002


## Interpolation

In [ ]:
# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('any', 'lunar', 'satellite'): 0}
bigram_probabilities = {( 'lunar', 'satellite'): 0.3}
unigram_probabilities = {'satellite': 0.4}


# the weights come from optimization on a validation set
lambda_1 = 0.8
lambda_2 = 0.15
lambda_3 = 0.05

# this is the input trigram we need to estimate
trigram = ('large', 'mars', 'satellite')

# find the last bigram and unigram of the input
bigram = trigram[1: 3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# in the production code, you would need to check if the probability n-gram dictionary contains the n-gram
probability_hat_trigram = lambda_1 * trigram_probabilities.get(trigram, 0)\
+ lambda_2 * bigram_probabilities.get(bigram, 0)\
+ lambda_3 * unigram_probabilities.get(unigram, 0)

print(f"estimated probability of the input trigram {trigram} is {probability_hat_trigram}")


besides the trigram ('large', 'mars', 'satellite') we also use bigram ('mars', 'satellite') and unigram (satellite)

estimated probability of the input trigram ('large', 'mars', 'satellite') is 0.020000000000000004


# Costruiamo un Language Model in grado di generare probabilità anche per combinazioni di n-gram appartenenti al vocabolario ma mai visti

## Convertiamo le parole fuori dal vocabolario nel token *UNK*

In [ ]:
train_news_texts_sample_processed_unk = []
for sentence in train_news_texts_sample_processed:
  new_sentence = []
  for word in sentence:
    if word not in vocabulary:
      new_sentence.append('UNK')
    else:
      new_sentence.append(word)
  train_news_texts_sample_processed_unk.append(new_sentence)

In [ ]:
train_news_texts_sample_processed_unk

[['<s>',
  '<s>',
  'any',
  'lunar',
  'satellite',
  'needs',
  'fuel',
  'to',
  'do',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'fuel',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  '.',
  '</s>'],
 ['<s>',
  '<s>',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'lunar',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  '.',
  '</s>'],
 ['<s>',
  '<s>',
  'any',
  'lunar',
  'satellite',
  'needs',
  'fuel',
  'to',
  'do',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'fuel',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  '.',
  '</s>']]

In [ ]:
n = 3
trigrams_list = [sentence_to_ngram(tokenized_sentence, n) for tokenized_sentence in train_news_texts_sample_processed_unk]
trigrams_list = [tuple(x) for y in trigrams_list for x in y]
count_trigrams = Counter(trigrams_list)

# Siccome dobbiamo applicare il k-smoothing, dobbiamo calcolare anche i conteggi dei bi-grammi, non solo dei tri-grammi
bigrams_list = [sentence_to_ngram(tokenized_sentence, n-1) for tokenized_sentence in train_news_texts_sample_processed_unk]
bigrams_list = [tuple(x) for y in bigrams_list for x in y]
count_bigrams = Counter(bigrams_list)


In [ ]:
def single_pass_ngram_probability_matrix_k_smoothing(k, n,  count_ngrams, count_n_minus_one_grams, vocabulary):
    """
    Crea la matrice di probabilità utilizzando i conteggi degli n-grammi.

    Args:
        k: fattore di smoothing da applicare
        n: numero degli n-grammi
        count_ngrams: Conteggio dei n-grammi presenti nel corpus
        count_n_minus_one_grams: Conteggio degli n-1-grammi presenti nel corpus
        vocabulary: lista di parole/token che fanno parte del nostro vocabolario

    Returns:
        n_minus_one_grams: lista di tutti i bigrammi, utilizzato come indice di riga della matrice
        vocabulary: lista di tutte le parole presenti nel corpus, utilizzato come indice di colonna
        count_matrix: pandas dataframe con i bigrammi prefixes come righe,
                      le parole del vocabolario come colonne
                      e il conteggio delle combinazioni bigramma/parola come valore
    """
    n_minus_one_grams = []
    prob_matrix_dict = defaultdict(dict)
    vocabulary_no_end_token = vocabulary.copy()
    vocabulary_no_end_token.remove('</s>')
    vocabulary_combinations = list(itertools.product(vocabulary_no_end_token, repeat=(n-1)))
    vocabulary_combinations = [x for x in vocabulary_combinations if not(x[0]!='<s>'
     and x[1]=='<s>')]

    vocabulary_no_start_token = vocabulary.copy()
    vocabulary_no_start_token.remove('<s>')
    vocabulary_size = len(vocabulary_no_start_token)

    for comb in vocabulary_combinations:
      n_gram_prefix_count = count_n_minus_one_grams.get(comb, 0)
      for word in vocabulary_no_start_token:
        n_gram_count = count_ngrams.get(comb + (word,), 0)
        prob_matrix_dict[comb,word] = add_k_smooting_probability(k, vocabulary_size, n_gram_count, n_gram_prefix_count)

    # convert the count_matrix to np.array to fill in the blanks
    prob_matrix = np.zeros((len(vocabulary_combinations), vocabulary_size))
    for ngram_key, ngram_count in prob_matrix_dict.items():
      prob_matrix[vocabulary_combinations.index(ngram_key[0]), \
                     vocabulary_no_start_token.index(ngram_key[1])] = ngram_count

    # np.array to pandas dataframe conversion
    prob_matrix = pd.DataFrame(prob_matrix, index=vocabulary_combinations, columns=vocabulary_no_start_token)
    return vocabulary_combinations, prob_matrix


In [ ]:
k=1
n_minus_one_grams, prob_matrix = single_pass_ngram_probability_matrix_k_smoothing(k, n,
                                                                                  count_trigrams,
                                                                                  count_bigrams,
                                                                                  vocabulary)

In [ ]:
prob_matrix

,fuel,lunar,.,</s>,any,satellite,needs,to,do,UNK
"(<s>, <s>)",0.076923,0.076923,0.076923,0.076923,0.230769,0.076923,0.076923,0.076923,0.076923,0.153846
"(<s>, fuel)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, lunar)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, .)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, any)",0.083333,0.250000,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
...,...,...,...,...,...,...,...,...,...,...
"(UNK, satellite)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(UNK, needs)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(UNK, to)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(UNK, do)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [ ]:
prob_matrix.sum(axis=1)

(<s>, <s>)          1.0
(<s>, fuel)         1.0
(<s>, lunar)        1.0
(<s>, .)            1.0
(<s>, any)          1.0
                   ... 
(UNK, satellite)    1.0
(UNK, needs)        1.0
(UNK, to)           1.0
(UNK, do)           1.0
(UNK, UNK)          1.0
Length: 91, dtype: float64

## Generiamo del testo utilizzando il nostro language model, con il nostro vocabolario e la possbilità di generare anche trigrammi non presenti nel nostro corpus di testi

In [ ]:
def generate_text(n: int, prob_matrix: pd.DataFrame, token_count: int, threshold_prob = 0.005, random_sampling: bool = False):
    """
    Funzione per generare del testo partendo dalla matrice di probabilità.

    Args:
        n: modello n-gramma da utilizzare
        prob_matrix: matrice di probabilità
        token_count: numero di token da generare
        threshold_prob: soglia di probabilità sopra la quale considerare i token
        random_sampling: booleano che dice se effettuare un sampling tra i token a probabilità non nulla oppure prendere sempre quello a probabilità massima

    Returns:
        bigrams: lista di tutti i bigrammi, utilizzato come indice di riga della matrice
        vocabulary: lista di tutte le parole presenti nel corpus, utilizzato come indice di colonna
        count_matrix: pandas dataframe con i bigrammi prefixes come righe,
                      le parole del vocabolario come colonne
                      e il conteggio delle combinazioni bigramma/parola come valore
    """
    context_queue = (n - 1) * ['<s>']
    result = []
    for _ in range(token_count):
      if random_sampling:
        if tuple(context_queue) in prob_matrix.index.tolist():
          nonzero_probs = (prob_matrix.loc[[tuple(context_queue)]]> threshold_prob).any()
          tokens_list = nonzero_probs.index[nonzero_probs].tolist()
          obj = random.sample(tokens_list, 1)[0]
        else:
          return ' '.join(result)
      else:
        obj = prob_matrix.loc[[tuple(context_queue)]].max().idxmax()
      result.append(obj)
      if n > 1:
          context_queue.pop(0)
          if obj == '.':
              context_queue = (n - 1) * ['<s>']
          else:
              context_queue.append(obj)
    return ' '.join(result)


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=False))

any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
any lunar satellite needs fuel to do UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True))

to lunar satellite satellite </s>
lunar lunar satellite </s>
fuel to lunar satellite </s>
do satellite to any needs needs any to fuel do needs fuel needs </s>
to UNK satellite to fuel needs . UNK fuel fuel </s>
fuel any </s>
do UNK fuel do any lunar UNK to satellite UNK lunar lunar do . </s>
UNK UNK needs satellite satellite any any to UNK satellite UNK satellite needs needs lunar . fuel satellite to fuel
</s>
lunar UNK satellite fuel do to UNK UNK fuel UNK </s>
